<a href="https://colab.research.google.com/github/marcellobenedetti92/programmazione-con-python/blob/main/progetto_finale_codice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions

### CSV check

In [ ]:
def csv_check(file_path):

  if not "store.csv" in os.listdir():
    print("Il file store.csv non è stato trovato.")
    print("Sto creando un file store.csv per te ...")

    # dataframe creation
    # non serve usare pandas, basta usare la funzione open
    # f = open(file_path, "w")
    # f.write("name, quantity,single_sale_price,single_pur_price,quantity_sold,tot_sold,tot_bought")
    # f.close()
    df = pd.DataFrame(columns=["name", "quantity", "single_sale_price", "single_pur_price", "quantity_sold", "tot_sold", "tot_bought"])
    df.to_csv(file_path, index = False)
    print("Il file store.csv è stato creato.\n")

### Add

In [ ]:
def add_products(file_path,df):

  # ask for input
  name = str(input("\nNome del prodotto: \n"))

  # if products not exist, check format
  if name not in df.values:
      
      quantity = int(input(f"\nQuantità: "))
      assert quantity >= 0, "Il valore deve essere positivo"

      single_sale_price = float(input(f"\nPrezzo di vendita: "))
      assert single_sale_price >= 0, "Il valore deve essere positivo"
      
      single_pur_price = float(input(f"\nPrezzo di acquisto: "))
      assert single_pur_price >= 0, "Il valore deve essere positivo"

      # put new products in last pandas data frame raw
      df = df.append({"name": name, "quantity": quantity, "single_sale_price": single_sale_price, "single_pur_price": single_pur_price}, ignore_index=True)


  else:

      quantity = int(input("Quantità: "))
      assert quantity >= 0, "Il valore deve essere positivo"

      index_prod = pd.Index(list(df["name"])).get_loc(name)
      prod_quantity = df["quantity"].iloc[index_prod]
      new_quantity = prod_quantity + quantity
      df.at[index_prod, "quantity"] = new_quantity

  print(f"\nAGGIUNTO: {quantity} X {name}\n")
  df.to_csv(file_path, index = False, na_rep=float(0))

### Sell

In [ ]:
def sell_products(file_path, df):

  try:

    name = input("\nNome del prodotto: ").lower()
    assert name in df.values, "Il prodotto non esiste nello store, provane un altro.\n"

    # products index finder
    # esistono soluzioni migliori per fare la ricerca
    # guarda qui https://stackoverflow.com/a/41255229/1694316

    prod_index = pd.Index(list(df["name"])).get_loc(name)
    prod_quantity = df["quantity"].iloc[prod_index]
    single_sale_price = float(df["single_sale_price"].iloc[prod_index])
    
    # check if quantity is in the store

    quantity_to_sell = int(input("\nQuantità: "))

    assert quantity_to_sell >= 0, "Il valore deve essere positivo"

    assert quantity_to_sell <= prod_quantity, "Non ci sono abbastanza prodotti"

    # sold and append new value 

    new_total_quantity = prod_quantity - quantity_to_sell 
    tot_price = quantity_to_sell * single_sale_price
    tot_list.append(tot_price)
    items_to_print.append([name, new_total_quantity, quantity_to_sell, single_sale_price])
      
    #add product

    add_more_product = input("\nAggiungere un altro prodotto? (si/no): ")

    if add_more_product.lower() == "si":
      sell_products(file_path, df)

    else: 
      print("\nVENDITA REGISTRATA")

      for p, nq, q, pr in items_to_print: # read sold element list and print
        index_p = pd.Index(list(df["name"])).get_loc(p)
        df.at[index_p, "quantity"] = nq
        df.at[index_p, "quantity_sold"] += q
        print(f"{q} X {p}: €{pr} \n")


      #print(items_to_print) per DEBUG

        
      tot = sum(tot_list) # calculate and print total sold
      print(f"Totale: {tot:.2f} €\n")
 
    df.to_csv(file_path, index = False)

    items_to_print.clear()
    tot_list.clear()

  except Exception as e:
    print(e,"\n")
    return sell_products(file_path, df)

### Profit

In [ ]:
def profit():

  gross_profit = sum(df["tot_sold"])
  net_profit = gross_profit - sum(df["tot_bought"])

  print(f"\nProfitto: lordo = €{gross_profit:.2f} netto = €{net_profit:.2f}\n")

### List

In [ ]:
def list_products(df):

  print ("\n{:<15} {:<15} {:<15}".format("PRODOTTO", "QUANTITA'", "PREZZO"))

  for pr, qu, pz in zip(df.name, df.quantity, df.single_sale_price):
    print("{:<15} {:<15} € {:<15}".format(pr, qu, pz))

  print("\n")

### Help

In [ ]:
def help():
  print("\nI comandi disponibili sono i seguenti: \n*aggiungi: aggiungi un prodotto al magazzino \n*elenca: elenca i prodotto in magazzino \n*vendita: registra una vendita effettuata \n*profitti: mostra i profitti totali \n*aiuto: mostra i possibili comandi \n*chiudi: esci dal programma\n")

### Total update

In [ ]:
def total_update(df, file_path):

  # update values in tot_sold
  tot_sold = round(df["quantity_sold"]*df["single_sale_price"],2)
  df["tot_sold"].update(list(tot_sold))

  # update values in tot_bought
  tot_bought = round((df["quantity_sold"]+df["quantity"])*df["single_pur_price"],2)
  df["tot_bought"].update(list(tot_bought))

  df.to_csv(file_path, index = False)

# Main Loop

In [ ]:
import os
import pandas as pd

cmd = None
items_to_print = []
tot_list = []
file_path = os.getcwd()+"/store.csv"
csv_check(file_path)

while cmd!="close".lower():

  df = pd.read_csv(file_path)
  total_update(df, file_path)

  cmd = input("Inserisci un comando: ")

  if cmd=="aggiungi":
    add_products(file_path,df)

  elif cmd=="vendita":
    sell_products(file_path,df)

  elif cmd=="profitti":
    profit()

  elif cmd=="elenca":
    list_products(df)

  elif cmd=="aiuto":
    help()

  elif cmd=="chiudi":
    print("\nBye bye") 
    # qui dovevi inserire un break per interrompere il programma, altrimenti il ciclo andrà avanti
  else:
    print("\nComando non valido. \nI comandi disponibili sono i seguenti:\n\n*aggiungi: aggiungi un prodotto al magazzino \n*elenca: elenca i prodotto in magazzino \n*vendita: registra una vendita effettuata \n*profitti: mostra i profitti totali \n*aiuto: mostra i possibili comandi \n*chiudi: esci dal programma\n")


AGGIUNTO: 20 X latte di soia


PRODOTTO        QUANTITA'       PREZZO         
latte di soia   20              € 1.4            



AGGIUNTO: 10 X tofu


Comando non valido. 
I comandi disponibili sono i seguenti:

*aggiungi: aggiungi un prodotto al magazzino 
*elenca: elenca i prodotto in magazzino 
*vendita: registra una vendita effettuata 
*profitti: mostra i profitti totali 
*aiuto: mostra i possibili comandi 
*chiudi: esci dal programma


AGGIUNTO: 5 X seitan


PRODOTTO        QUANTITA'       PREZZO         
latte di soia   20              € 1.4            
tofu            10              € 4.19           
seitan          5               € 5.49           



VENDITA REGISTRATA
5 X latte di soia: €1.4 

2 X tofu: €4.19 

Totale: 15.38 €


PRODOTTO        QUANTITA'       PREZZO         
latte di soia   15              € 1.4            
tofu            8               € 4.19           
seitan          5               € 5.49           



VENDITA REGISTRATA
5 X seitan: €5.49 

Totale